# Adaptative Split-net

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


## Loading the data

In [1]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.datasets import cifar10

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=1974)

In [0]:
train_features = X_train.reshape(X_train.shape[0], 32, 32, 3)/255
vali_features = X_val.reshape(X_val.shape[0], 32, 32, 3)/255
test_features = X_test.reshape(X_test.shape[0], 32, 32, 3)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

## Defining coarse labels

In [0]:
dict_coarse1 = {0: 0,
              1: 0,
              2: 1,
              3: 1,
              4: 1,
              5: 1,
              6: 1,
              7: 1,
              8: 0,
              9: 0}

dict_coarse2 = {0: 0,
              1: 1,
              2: 3,
              3: 4,
              4: 6,
              5: 4,
              6: 5,
              7: 6,
              8: 2,
              9: 1}

y_train_coarse1 = np.vectorize(dict_coarse1.get)(y_train)
y_val_coarse1 = np.vectorize(dict_coarse1.get)(y_val)
y_test_coarse1 = np.vectorize(dict_coarse1.get)(y_test)

y_train_coarse2 = np.vectorize(dict_coarse2.get)(y_train)
y_val_coarse2 = np.vectorize(dict_coarse2.get)(y_val)
y_test_coarse2 = np.vectorize(dict_coarse2.get)(y_test)

y_train_c_cat1 = np_utils.to_categorical(y_train_coarse1)
y_val_c_cat1 = np_utils.to_categorical(y_val_coarse1)
y_test_c_cat1 = np_utils.to_categorical(y_test_coarse1)

y_train_c_cat2 = np_utils.to_categorical(y_train_coarse2)
y_val_c_cat2 = np_utils.to_categorical(y_val_coarse2)
y_test_c_cat2 = np_utils.to_categorical(y_test_coarse2)

## Training the model

In [0]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [0]:
class TimingCallback(keras.callbacks.Callback):
  """Callback that saves the time elapsed of each epoch to the log.
  """  
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [0]:
class AdaptativeLossWeightsModifier3Vars(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma, decay_rate=0.5):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
    self.decay_rate = decay_rate
    self.offset_epoch = 0
    self.par_reduce = [0, 1]
  
  def calculate_exponential(self, ratio, decay_rate, epoch):
    return np.exp(-ratio*decay_rate*epoch)

  def on_epoch_end(self, epoch, logs={}):
    list_vars = [K.eval(self.alpha), K.eval(self.beta), K.eval(self.gamma)]
    if epoch < 1:
      pass
    
    else:
      loss_coarse1 = self.model.history.history['predictions_coarse1_loss'][-1]
      loss_coarse2 = self.model.history.history['predictions_coarse2_loss'][-1]
      loss_fine = self.model.history.history['predictions_fine_loss'][-1]
      
      losses_classes = [loss_coarse1, loss_coarse2, loss_fine]
      ratio = losses_classes[self.par_reduce[0]] / losses_classes[self.par_reduce[1]] *(epoch + 1 - self.offset_epoch) 
      
      decaying = self.calculate_exponential(ratio, self.decay_rate/(max(self.par_reduce)**2), epoch)
      increasing = 1 - decaying
    
    
      if (1 - increasing) < 0.3 and (self.par_reduce[1] < len(list_vars) - 1):
        list_vars[self.par_reduce[0]] = 0
        list_vars[self.par_reduce[1]] = 1
        self.par_reduce = [i + 1 for i in self.par_reduce]
        self.offset_epoch = self.offset_epoch + epoch

      else:
       list_vars[self.par_reduce[0]] = decaying
       list_vars[self.par_reduce[1]] = increasing

        
      K.set_value(self.alpha, list_vars[0])
      K.set_value(self.beta, list_vars[1])
      K.set_value(self.gamma, list_vars[2])
        
      print('Changing loss weights to: coarse1 = {}, coarse2 = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta), K.eval(self.gamma)))
    
    logs['alpha'] = K.eval(self.alpha)
    logs['beta'] = K.eval(self.beta) 
    logs['gamma'] = K.eval(self.gamma) 

In [0]:
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)

num_classes_coarse1 = 2
num_classes_coarse2 = 7
num_classes_fine = 10

img_input = Input(shape=input_shape, name='input')


x = Conv2D(128, (3, 3), activation='relu', name='block1_conv1', padding='same')(img_input)

x = Conv2D(128, (3, 3), activation='relu', name='block1_conv2', padding='same')(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)

x = Conv2D(128, (3, 3), activation='relu', name='block1_conv3', padding='same')(x)
x = MaxPooling2D((2, 2), name='block3_pool')(x)

x = Flatten(name='flatten')(x)
x = Dense(512, name='fc_1')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(512, name='fc_2')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

coarse_pred1 = Dense(num_classes_coarse1, activation='softmax', name='predictions_coarse1')(x)
coarse_pred2 = Dense(num_classes_coarse2, activation='softmax', name='predictions_coarse2')(x)
fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(x)

In [14]:
model = Model(inputs=img_input, outputs= [coarse_pred1, coarse_pred2, fine_pred], name='adaptative_split_net')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 32, 32, 128)  3584        input[0][0]                      
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 32, 32, 128)  147584      block1_conv1[0][0]               
__________________________________________________________________________________________________
block2_pool (MaxPooling2D)      (None, 16, 16, 128)  0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block1_con

In [0]:
alpha = K.variable(value=0.9, dtype="float32", name="alpha") 
beta = K.variable(value=0.1, dtype="float32", name="beta") 
gamma = K.variable(value=0, dtype="float32", name="gamma") 

adam = optimizers.Adam(lr=0.00025)

model.compile(loss='categorical_crossentropy', 
              optimizer=adam,
              loss_weights=[alpha, beta, gamma],
              metrics=['accuracy'])

In [0]:
clr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
change_lw = AdaptativeLossWeightsModifier3Vars(alpha, beta, gamma, decay_rate=0.5)
csv_cb = CSVLogger('drive/TCC-ITAU/cifar-10/training-data/split-baseline-net/adaptative-split-baseline-net-simple.csv', separator=',', append=False)


cbks = [clr_cb, time_cb, change_lw, csv_cb]

In [17]:
epochs = 50
start_time = time()
training = model.fit(train_features, [y_train_c_cat1, y_train_c_cat2, y_train_cat],
                     validation_data=(vali_features, [y_val_c_cat1, y_val_c_cat2, y_val_cat]),
                     epochs=epochs,
                     batch_size=256, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 25s 625us/step - loss: 0.4933 - predictions_coarse1_loss: 0.3608 - predictions_coarse2_loss: 1.6858 - predictions_fine_loss: 3.1128 - predictions_coarse1_acc: 0.8599 - predictions_coarse2_acc: 0.3881 - predictions_fine_acc: 0.1018 - val_loss: 0.3557 - val_predictions_coarse1_loss: 0.2666 - val_predictions_coarse2_loss: 1.1573 - val_predictions_fine_loss: 2.8589 - val_predictions_coarse1_acc: 0.8999 - val_predictions_coarse2_acc: 0.5590 - val_predictions_fine_acc: 0.1070
Epoch 2/50
40000/40000 [==============================] - 23s 574us/step - loss: 0.3370 - predictions_coarse1_loss: 0.2340 - predictions_coarse2_loss: 1.2634 - predictions_fine_loss: 3.1522 - predictions_coarse1_acc: 0.9098 - predictions_coarse2_acc: 0.5237 - predictions_fine_acc: 0.0990 - val_loss: 0.2895 - val_predictions_coarse1_loss: 0.1989 - val_predictions_coarse2_loss: 1.1051 - val_predictions_fine_loss: 2.

In [18]:
model.evaluate(test_features, [y_test_c_cat1,y_test_c_cat2, y_test_cat])

10000/10000 [==============================] - 3s 327us/step


[1.0471150638580322,
 0.12046884723305702,
 0.5408105954885483,
 1.0471150638580322,
 0.9532,
 0.8227,
 0.7741]

In [19]:
print('Total training time: {}'.format(training_time))

Total training time: 1179.6658787727356


In [20]:
model.save('drive/TCC-ITAU/cifar-10/models/adaptative-split-net.h5')

TypeError: ignored